# `swift_too` module

## Swift_QueryJob example - Querying and fetching the results of an existing job

### API version = 1.2, `swifttools` version 3.0

#### Author: Jamie A. Kennea (Penn State)

If you wish to retrieve the results of a previously job, this is the task of the QueryJob class. All you need for this class is your username, shared secret and the job number which is reported back when the job is submitted either with `submit()` or `queue()` methods. The jobnumber can be found in the property `jobnumber`.

In [1]:
from swifttools.swift_too import QueryJob, VisQuery
from time import sleep

Previously we would have defined here our username and shared_secret, however in `swifttools 2.2`, for most requests you can skip this and they will default to an anonymous user. For this notebook we'll use the anonymous login to make things simpler.

First let's submit a simple job so we can get an example jobnumber. In this case I'm just going to do something that isn't too strenuous on the system, let's see if Sgr A* is currently visible, or when it is next visible.

In [2]:
vq = VisQuery()
vq.name = "Crab"
vq.hires = (True,)
vq.length = 1
if vq.queue():
    print(f"Queued job successfully! Jobnumber = {vq.status.jobnumber}")
else:
    print(f"Error: {vq.status.errors}")

Queued job successfully! Jobnumber = 578328


Note that the job has been submitted, but because we used the `queue()` method, it may not have completed yet. If you wait a bit, it will be, but the next step can be run straight away to demonstrate what happens if processing isn't complete.



OK let's see how our job is doing, because the purpose of the QueryJob class is to allow us to fetch a job later, or maybe ASAP. Maybe you don't want to hang around waiting for a bunch of jobs to complete, submit some, have some coffee, come back and fetch, or just want to query the result of an existing job.

In [3]:
qj = QueryJob()
qj.jobnumber = vq.status.jobnumber
qj.queue()

print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")

Current status of job #578328: Queued


Note that for the class `QueryJob`, as it does not take many parameters, you can also pass parameters as arguments. If you do this, then the request will be submitted on invokation, so there's no need use the `submit()` method.

In [4]:
while not qj:
    qj = QueryJob(jobnumber=vq.status.jobnumber)
    print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")
    sleep(1)

Current status of job #578328: Accepted


The `submit` (or argument form) should return `True` if the job has been fetched. It'll return `False` for a number of reasons, firstly if the job is still *Queued* (waiting to be processed) or *Processing*, the status will indicate this. Also the status can be *Rejected*, in the case where you have given bad parameters, or the job no longer exists.


The above "while" will keep polling the job until it is complete. We put in the sleep between requests so as to not over spam the TOO_API server.

OK let's assume that the above now reports "Current status of job #XXX: Accepted", which indicates a successful `QueryJob` call of a completed job. Let's see what our QueryJob class data now looks like.

In [5]:
qj

username,anonymous
jobnumber,578328
fetchresult,True
status,Accepted
errors,[]
warnings,[]
timestamp,2022-09-23 10:27:11
began,2022-09-23 10:27:11
completed,2022-09-23 10:27:11
result,Swift_VisQuery object


You can see the data inside the class here presented in table form. The the more useful value here is `result`, which contains the result of the job. In the Python api, this will be a `Swift_VisQuery` object. You can display the result of the job like this:

In [6]:
qj.result

2022-09-23 10:27:00,2022-09-23 11:08:00,0:41:00
2022-09-23 12:03:00,2022-09-23 12:42:00,0:39:00
2022-09-23 13:38:00,2022-09-23 14:19:00,0:41:00
2022-09-23 15:14:00,2022-09-23 15:55:00,0:41:00
2022-09-23 16:49:00,2022-09-23 17:30:00,0:41:00
2022-09-23 18:24:00,2022-09-23 19:06:00,0:42:00
2022-09-23 20:00:00,2022-09-23 20:41:00,0:41:00
2022-09-23 21:35:00,2022-09-23 22:17:00,0:42:00
2022-09-23 23:11:00,2022-09-23 23:52:00,0:41:00
2022-09-24 00:46:00,2022-09-24 01:27:00,0:41:00
2022-09-24 02:22:00,2022-09-24 03:03:00,0:41:00


Other parameters that might be of interest are the `timestamp`, `began` and `completed` values, which allow you to find out when the job was submitted, when it started processing and when it finished. For example:

In [7]:
print(
    f"Job was submitted at {qj.timestamp}, was queued for {(qj.began - qj.timestamp).seconds}s and took {(qj.completed - qj.began).seconds}s to process."
)

Job was submitted at 2022-09-23 10:27:11, was queued for 0s and took 0s to process.


As usual, times reported back are UT. That is it! If all is correct, then the following show the upcoming or current *Swift* visibility window for Sgr A*!

In [8]:
_ = [print(win) for win in qj.result]

2022-09-23 10:27:00 - 2022-09-23 11:08:00 (0:41:00)
2022-09-23 12:03:00 - 2022-09-23 12:42:00 (0:39:00)
2022-09-23 13:38:00 - 2022-09-23 14:19:00 (0:41:00)
2022-09-23 15:14:00 - 2022-09-23 15:55:00 (0:41:00)
2022-09-23 16:49:00 - 2022-09-23 17:30:00 (0:41:00)
2022-09-23 18:24:00 - 2022-09-23 19:06:00 (0:42:00)
2022-09-23 20:00:00 - 2022-09-23 20:41:00 (0:41:00)
2022-09-23 21:35:00 - 2022-09-23 22:17:00 (0:42:00)
2022-09-23 23:11:00 - 2022-09-23 23:52:00 (0:41:00)
2022-09-24 00:46:00 - 2022-09-24 01:27:00 (0:41:00)
2022-09-24 02:22:00 - 2022-09-24 03:03:00 (0:41:00)
2022-09-24 03:57:00 - 2022-09-24 04:38:00 (0:41:00)
2022-09-24 05:33:00 - 2022-09-24 06:14:00 (0:41:00)
2022-09-24 07:08:00 - 2022-09-24 07:49:00 (0:41:00)
2022-09-24 08:43:00 - 2022-09-24 09:25:00 (0:42:00)
2022-09-24 10:19:00 - 2022-09-24 10:27:00 (0:08:00)


That first date is going to be the first visibility date, which if the target is visible now should be essentially the same as `timestamp` in `QueryJob`, or sometime in the future if the target was constrained when the job was submitted. 

In [9]:
print(qj.result[0][0] - qj.timestamp)

-1 day, 23:59:49


Note due to the weird way datetime displays negative timediffs, you might see something like `-1 day, 23:59:19` if the value is negative!